# Automated Profiling In PyTorch Lightning 

This example (which utilizes the same problem set-up as in "complex_objectve_function_example.py) shows how one can easily use a Torch Profiler with PyTorch Lightning to view bottlenecks and performance benchmarking. 



### Imports

In [1]:
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader

import neuromancer as nm
from neuromancer.dataset import DictDataset

import lightning.pytorch as pl 
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from torch.utils.data import Dataset, DataLoader

from neuromancer.trainer import Trainer, LitTrainer
from neuromancer.problem import Problem, LitProblem
from neuromancer.constraint import variable
from neuromancer.dataset import DictDataset, LitDataModule
from neuromancer.loss import PenaltyLoss
from neuromancer.modules import blocks
from neuromancer.system import Node

### Problem and Data Definition

In [10]:
# Define the data setup function
def data_setup_function(exp_returns): 
        p_low, p_high = max(min(exp_returns),0), max(exp_returns)
        data_train = DictDataset({"p": torch.FloatTensor(1000, 1).uniform_(p_low, p_high)}, name='train')
        data_test = DictDataset({"p": torch.FloatTensor(100, 1).uniform_(p_low, p_high)}, name='test')
        data_dev = DictDataset({"p": torch.FloatTensor(100, 1).uniform_(p_low, p_high)}, name='dev')
        return data_train, data_dev, data_test, 32



num_vars = 5 #set to 2 for performance reasons

# expected returns
exp_returns = np.random.uniform(0.002, 0.01, num_vars)
print("Expected Returns:")
print(exp_returns)

# covariance matrix
A = np.random.rand(num_vars, num_vars)
# positive semi-definite matrix
cov_matrix = A @ A.T / 1000
print("Covariance Matrix:")
print(cov_matrix)


# parameters
p = nm.constraint.variable("p")
# variables
x = nm.constraint.variable("x")

# objective function
f = sum(cov_matrix[i, j] * x[:, i] * x[:, j] for i in range(num_vars) for j in range(num_vars))
obj = [f.minimize(weight=1.0, name="obj")]

# constraints
constrs = []
# constr: 100 units
con = 100 * (sum(x[:, i] for i in range(num_vars)) == 1)
con.name = "c_units"
constrs.append(con)
# constr: expected return
con = 100 * (sum(exp_returns[i] * x[:, i] for i in range(num_vars)) >= p[:, 0])
con.name = "c_return"
constrs.append(con)

# define neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=1, outsize=num_vars, bias=True,
                        linear_map=nm.slim.maps["linear"], nonlin=nn.ReLU, hsizes=[5]*4)
# solution map from model parameters: sol_map(p) -> x
sol_map = nm.system.Node(func, ["p"], ["x"], name="smap")
# trainable components
components = [sol_map]

# merit loss function
loss = nm.loss.PenaltyLoss(obj, constrs)
# problem
problem = nm.problem.Problem(components, loss)

Expected Returns:
[0.00705232 0.00291352 0.00902834 0.00806446 0.00449609]
Covariance Matrix:
[[0.0023143  0.00113818 0.00231431 0.00112589 0.00155372]
 [0.00113818 0.00083598 0.00163794 0.00078066 0.00101279]
 [0.00231431 0.00163794 0.00333726 0.00137955 0.00197275]
 [0.00112589 0.00078066 0.00137955 0.00094245 0.00101531]
 [0.00155372 0.00101279 0.00197275 0.00101531 0.00162734]]


### Train Using Lightning and Profile

We will use the "simple" profiler for viewing end-to-end runtime. For bottleneck analysis, the recommendation is to use "pytorch" profiler. For more profiling options please refer to: 

https://pytorch-lightning.readthedocs.io/en/1.5.10/advanced/profiler.html

In [11]:
# training
lr = 0.001  # step size for gradient descent

# set adamW as optimizer
optimizer = torch.optim.AdamW(problem.parameters(), lr=lr)


# Define lightning data module
lit_data_module = LitDataModule(data_setup_function=data_setup_function, exp_returns=exp_returns)

# Define lightning trainer. We use GPU acceleration utilizing 2 GPUS. We tell Lightning to 
# distribute training parallely (strategy=ddp)
lit_trainer = LitTrainer(epochs=1, accelerator="cpu", dev_metric='dev_loss', profiler='pytorch')

# Train problem to the data module
best_problem_weights = lit_trainer.fit(problem, lit_data_module)

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


{'exp_returns': array([0.00705232, 0.00291352, 0.00902834, 0.00806446, 0.00449609])}


/home/birm560/miniconda3/envs/neuromancer3/lib/python3.10/site-packages/lightning/pytorch/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...
